In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install pytorch-transformers
!pip install transformers
!pip install nltk


     |████████████████████████████████| 184kB 7.5MB/s 
     |████████████████████████████████| 133kB 8.5MB/s 
     |████████████████████████████████| 1.2MB 11.6MB/s 
     |████████████████████████████████| 870kB 27.1MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 7.4MB 36.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=48c26de74e29e570aaea1e9ef6d8513dfa17638b4bee04f152464082d915c1aa
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
ERROR: botocore 1.20.49 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 2.2MB 9.1MB/s 
     |████████████████████████████████| 3.3MB 55.0MB/s 


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install tweet-preprocessor

In [ ]:
pip install stopwordsiso

     |████████████████████████████████| 81kB 5.3MB/s 


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW
import nltk
#import stopwordsiso
#from stopwordsiso import stopwords
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
#from nltk.corpus import stopwords
#stop_words = set(stopwords("id"))

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
# % matplotlib inline


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [ ]:
MAX_LEN = 128

In [ ]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/new_train.csv')
train['id'] = range(1, len(train) + 1)
train['label'] = train['label'].astype(int)

In [ ]:
train.head()

,text,label,id
0,据说 卡塔尔 要 退出 石油输出国组织 ， 这是 川普 退群潮 的 连锁反应 ， 也 是 世...,0,1
1,早上 没 起床 先 各 渠道 看看 与 川普 在 阿根廷 的 会面 。 留给 中国 国家足球...,1,2
2,面对 加拿大 及 美国 的 无理 行为 ， 中国 也 可以 （ 无 任何理由 ） 把 加拿大...,0,3
3,如果 特朗普 宣布 美国 对 2000 亿 中国 出口 美国 商品 征收 10% 的 关税 ...,1,4
4,两个 白日梦 什么 时候 能 像 宣传 禁毒 一样 宣传 反 强奸 什么 时候 防空警报 能...,0,5


In [ ]:
val_df = pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/new_dev.csv')
val_df['id'] = range(1, len(val_df) + 1)
val_df['label'] = val_df['label'].astype(int)

In [ ]:
test_df = pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/test.csv')
test_df['id'] = range(1, len(val_df) + 1)

In [ ]:
#df, test_df = np.split(train.sample(frac=1), [int(.8*len(train))])

#df = df.reset_index(drop=True)
#val_df = val_df.reset_index(drop=True)
#test_df = test_df.reset_index(drop=True)


print("Size of training set: {}".format(len(train)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 1512
Size of validation set: 189
Size of testing set: 189


In [ ]:
p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess(row, lemmatizer, stemmer):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  tokenization = nltk.word_tokenize(text)     
  #tokenization = [w for w in tokenization if not w in stop_words]
#   text = ' '.join([porter_stemmer.stem(w) for w in tokenization])
#   text = ' '.join([lemmatizer.lemmatize(w) for w in tokenization])
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [ ]:
def map_label(row):
  return 0 if row['label']==1 else 1

train['label_encoded'] = train.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [ ]:
train_sentences = train.text.values
val_sentences = val_df.text.values
test_sentences = test_df.text.values

train_labels = train.label_encoded.values
val_labels = val_df.label_encoded.values


In [ ]:
# tokenizer_sentences = np.array(list(df.tweet.values) + list(val_df.tweet.values))

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('hfl/chinese-xlnet-base', do_lower_case=False)


In [ ]:
def Encode_TextWithAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=True):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids'],encoded_dict['attention_mask']

def Encode_TextWithoutAttention(sentence,tokenizer,maxlen,padding_type='max_length',attention_mask_flag=False):
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maxlen, truncation=True, padding=padding_type, return_attention_mask=attention_mask_flag)
    return encoded_dict['input_ids']

def get_TokenizedTextWithAttentionMask(sentenceList, tokenizer):
    token_ids_list,attention_mask_list = [],[]
    for sentence in sentenceList:
        token_ids,attention_mask = Encode_TextWithAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
        attention_mask_list.append(attention_mask)
    return token_ids_list,attention_mask_list

def get_TokenizedText(sentenceList, tokenizer):
    token_ids_list = []
    for sentence in sentenceList:
        token_ids = Encode_TextWithoutAttention(sentence,tokenizer,MAX_LEN)
        token_ids_list.append(token_ids)
    return token_ids_list

In [ ]:
train_token_ids,train_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(train_sentences,tokenizer))
val_token_ids,val_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(val_sentences,tokenizer))
test_token_ids,test_attention_masks = torch.tensor(get_TokenizedTextWithAttentionMask(test_sentences,tokenizer))

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [ ]:
batch_size = 16

train_data = TensorDataset(train_token_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_token_ids, val_attention_masks, val_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_token_ids, test_attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [ ]:
model = XLNetForSequenceClassification.from_pretrained("hfl/chinese-xlnet-base", num_labels=2).cuda()



Some weights of the model checkpoint at hfl/chinese-xlnet-base were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-xlnet-base and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
train_loss_set = []
best_val_accuracy = 0.50
directory_path = '/content/gdrive/MyDrive/4th_sem/dataset/Censorship/train_test'
epochs = 20

for _ in trange(epochs, desc="Epoch"):
  
  model.train()
  
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    loss.backward()
    optimizer.step()
        
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
  model.eval()

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  Validation_Accuracy = (eval_accuracy/nb_eval_steps)
  if(Validation_Accuracy >= best_val_accuracy):
    torch.save(model.state_dict(), directory_path+'/models/XLNet_base_model.ckpt')
    best_val_accuracy = Validation_Accuracy
    print('Model Saved')

    


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Train loss: 0.6985552662297299
Validation Accuracy: 0.5953525641025641


Epoch:   5%|▌         | 1/20 [00:45<14:22, 45.37s/it]

Model Saved
Train loss: 0.6656108144082521
Validation Accuracy: 0.6334134615384616


Epoch:  10%|█         | 2/20 [01:32<13:44, 45.79s/it]

...

Validation Accuracy: 0.655448717948718
Train loss: 0.017531370828357085


Epoch:  95%|█████████▌| 19/20 [14:41<00:46, 46.16s/it]

Validation Accuracy: 0.6658653846153846
Train loss: 0.005472488822277611
Validation Accuracy: 0.687900641025641


Epoch: 100%|██████████| 20/20 [15:29<00:00, 46.50s/it]

Model Saved


In [ ]:
print(train.dtypes)

text             object
label             int64
id                int64
label_encoded     int64
dtype: object


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import preprocessor as p

from transformers import XLNetTokenizer, XLNetForSequenceClassification,BertTokenizer, BertForSequenceClassification, DebertaTokenizer, DebertaForSequenceClassification, RobertaTokenizerFast, RobertaForSequenceClassification, ElectraTokenizer, ElectraForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla T4'

In [ ]:
train=pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/new_train.csv')
train['id'] = range(1, len(train) + 1)
train['label'] = train['label'].astype(int)

In [ ]:
val_df = pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/new_dev.csv')
val_df['id'] = range(1, len(val_df) + 1)
val_df['label'] = val_df['label'].astype(int)

In [ ]:
test_df = pd.read_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/test.csv')
test_df['id'] = range(1, len(val_df) + 1)

In [ ]:
print(train.dtypes)

text     object
label     int64
id        int64
dtype: object


In [ ]:
#df, test_df = np.split(train.sample(frac=1), [int(.8*len(train))])

#df = df.reset_index(drop=True)
#val_df = val_df.reset_index(drop=True)
#test_df = test_df.reset_index(drop=True)
test_label_df = test_df


print("Size of training set: {}".format(len(train)))
print("Size of validation set: {}".format(len(val_df)))
print("Size of testing set: {}".format(len(test_df)))

Size of training set: 1512
Size of validation set: 189
Size of testing set: 189


In [ ]:
def preprocess(row):
  text = row['text']
  # text = text.strip('\xa0')
  text = p.clean(text)
  # text = re.sub(r'\([0-9]+\)', '', text).strip()    
  return text


In [ ]:
def map_label(row):
  return 0 if row['label']==1 else 1

train['label_encoded'] = train.apply(lambda x: map_label(x), 1)
val_df['label_encoded'] = val_df.apply(lambda x: map_label(x), 1)
# test_df['label_encoded'] = test_df.apply(lambda x: map_label(x), 1)


In [ ]:
train_sentences = train.text.values
train_token_ids = train.id.values
val_sentences = val_df.text.values
val_token_ids = val_df.id.values
test_sentences = test_df.text.values
test_token_ids = test_df.id.values


In [ ]:
train_sentences = [sentence + " [SEP] [CLS]" for sentence in train_sentences]
train_labels = train.label_encoded.values
val_sentences = [sentence + " [SEP] [CLS]" for sentence in val_sentences]
val_labels = val_df.label_encoded.values
test_sentences = [sentence + " [SEP] [CLS]" for sentence in test_sentences]


In [ ]:
MAX_LEN = 128
batch_size = 64


In [ ]:
def get_dataloader(network, train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids):
  dic = {"XLNET": "hfl/chinese-xlnet-base", "BERT": "bert-base-uncased", "ROBERT": "roberta-base", "XLM-ROBERT": "xlm-roberta-base", "DeBERTa": "microsoft/deberta-base", "Electra": "google/electra-base-discriminator"}
  if network=='XLNET':
    tokenizer = XLNetTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='DeBERTa':
    tokenizer = DebertaTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='BERT':
    tokenizer = BertTokenizer.from_pretrained(dic[network], do_lower_case=True)
  elif network=='Electra':
    tokenizer = ElectraTokenizer.from_pretrained(dic[network], do_lower_case=True)
  else:
    tokenizer = RobertaTokenizerFast.from_pretrained(dic[network], do_lower_case=True)

  tokenized_train_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
  print ("Tokenize the first sentence:")
  print (tokenized_train_texts[0])

  tokenized_val_texts = [tokenizer.tokenize(sent) for sent in val_sentences]
  tokenized_test_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
  input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train_texts]
  input_val_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_val_texts]
  input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test_texts]

  input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_val_ids = pad_sequences(input_val_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  # Create attention masks
  train_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_train_ids:
    seq_mask = [float(i>0) for i in seq]
    train_attention_masks.append(seq_mask)
  train_masks = np.array(train_attention_masks)

  # Create attention masks
  val_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_val_ids:
    seq_mask = [float(i>0) for i in seq]
    val_attention_masks.append(seq_mask)
  validation_masks = np.array(val_attention_masks)

  # Create attention masks
  test_attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_test_ids:
    seq_mask = [float(i>0) for i in seq]
    test_attention_masks.append(seq_mask)
  test_masks = np.array(test_attention_masks)

  train_inputs = torch.tensor(input_train_ids)
  validation_inputs = torch.tensor(input_val_ids)
  test_inputs = torch.tensor(input_test_ids)
  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(val_labels)
  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)
  test_masks = torch.tensor(test_masks)

  train_data = TensorDataset(torch.tensor(train_token_ids), train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  validation_data = TensorDataset(torch.tensor(val_token_ids), validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  test_data = TensorDataset(torch.tensor(test_token_ids), test_inputs, test_masks)
  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

  return train_dataloader, validation_dataloader, test_dataloader


In [ ]:
train_dataloader_XLNET, val_dataloader_XLNET, test_dataloader_XLNET = get_dataloader("XLNET", train_sentences, val_sentences, train_labels, val_labels, test_sentences, train_token_ids, val_token_ids, test_token_ids)

Tokenize the first sentence:
['▁', '据说', '▁', '卡塔尔', '▁', '要', '▁', '退出', '▁', '石油', '输出', '国', '组织', '▁', ',', '▁这是', '▁', '川', '普', '▁', '退', '群', '潮', '▁', '的', '▁', '连', '锁', '反应', '▁', ',', '▁', '也', '▁', '是', '▁', '世界', '秩序', '▁', '和', '▁', '格局', '▁', '重', '构', '▁', '的', '▁', '机', '缘', '▁', '。', '▁', '对', '▁', '石油', '▁', '消费', '国', '▁', '应该', '▁', '是', '▁', '好', '事', '▁[', 's', 'ep', ']', '▁[', 'cl', 's', ']']


In [ ]:
model3 = XLNetForSequenceClassification.from_pretrained("hfl/chinese-xlnet-base", num_labels=2)
directory_path = '/content/gdrive/MyDrive/4th_sem/dataset/Censorship/train_test/models'
model3.load_state_dict(torch.load(directory_path+'/XLNet_base_model.ckpt'))
model3.eval()
model3.cuda()

Some weights of the model checkpoint at hfl/chinese-xlnet-base were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-xlnet-base and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for p

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

# New Section

In [ ]:
def get_model_preds_labels(model, dataloader, mode='train'):
  data_vectors = []
  labels = []
  ids = []

  with torch.no_grad():
      correct = 0
      total = 0
      for i, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)
        if mode=='test':
            token_ids, b_input_ids, b_input_mask = batch
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            for point, tk_id in zip(F.softmax(outputs.logits).tolist(), token_ids.tolist()):
              data_vectors.append(point)       
              ids.append(tk_id)     
        else:
          # Unpack the inputs from our dataloader
          token_ids, b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          for point, lab, tk_id in zip(F.softmax(outputs.logits).tolist(), b_labels.tolist(), token_ids.tolist()):
            data_vectors.append(point)
            labels.append(lab)
            ids.append(tk_id)

  if mode=='test':
    return data_vectors, ids
  else:
    return data_vectors, labels, ids

#Train Data

In [ ]:
m3, _, train_ids3 = get_model_preds_labels(model3, train_dataloader_XLNET)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in m3]
train_pred_df = pd.DataFrame({'id': train_ids3, 'predicted_label': pred_labels})
train_pred_df = train[["id", "label"]].merge(train_pred_df, on='id', how='left')
train_pred_df.head()

,id,label,predicted_label
0,1,0,0
1,2,1,1
2,3,0,0
3,4,1,1
4,5,0,0


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score


In [ ]:
confusion_matrix(train_pred_df['label'].values, train_pred_df['predicted_label'].values)

array([[742,   8],
       [  1, 761]])

In [ ]:
print(accuracy_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values))
print(precision_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(recall_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))
print(f1_score(train_pred_df['label'].values, train_pred_df['predicted_label'].values, average='micro'))


0.9940476190476191
0.9940476190476191
0.9940476190476191
0.9940476190476191


#Validation Data

In [ ]:
v3, _, v_ids3 = get_model_preds_labels(model3, val_dataloader_XLNET)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
val_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in v3]
val_pred_df = pd.DataFrame({'id': v_ids3, 'predicted_label': val_pred_labels})
val_pred_df = val_df[["id", "label"]].merge(val_pred_df, on='id', how='left')
val_pred_df.head()



,id,label,predicted_label
0,1,1,1
1,2,1,1
2,3,1,0
3,4,0,0
4,5,0,1


In [ ]:
confusion_matrix(val_pred_df['label'].values, val_pred_df['predicted_label'].values)

array([[60, 36],
       [33, 60]])

In [ ]:
print(accuracy_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values))
print(precision_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(recall_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))
print(f1_score(val_pred_df['label'].values, val_pred_df['predicted_label'].values, average='micro'))

0.6349206349206349
0.6349206349206349
0.6349206349206349
0.6349206349206349


#Test Data

In [ ]:
t3, t_ids3 = get_model_preds_labels(model3, test_dataloader_XLNET, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [ ]:
test_pred_labels = [1 if np.argmax(np.array([each]), 1)[0]==0 else 0 for each in t3]
test_pred_df = pd.DataFrame({'id': t_ids3, 'predicted_label': test_pred_labels})
test_pred_df = test_df[["id", "text"]].merge(test_pred_df, on='id', how='left')
test_pred_df.head()

,id,text,predicted_label
0,1,20 . 一位 机械工程 专家 讲过 这样 一件 事 ： “ 文革 ” 中 ， 他 在 某地...,1
1,2,警惕 看不到 内心 的 人 ， 虚荣 的 人 ， 心狠 的 人 ， 没有 是非观 的 人 ，...,1
2,3,这个 国在 计划生育 的 时候 对 人 都 能 下 杀手 ， 何况 现在 杭州 政策 杀狗 ...,1
3,4,罗伯特 . 所罗门 《 大 问题 — — 简明 哲学 导论 》 在 我们 这样 一个多元 的...,1
4,5,特朗普 ， 输 了 ！ O 特朗普 ， 输 了 ！,1


In [ ]:
test_pred_df = test_pred_df.rename(columns={'id': 'Tweet id', 'text': 'Tweet', 'predicted_label':'label'})

In [ ]:
test_pred_df.head()

,Tweet id,Tweet,label
0,1,20 . 一位 机械工程 专家 讲过 这样 一件 事 ： “ 文革 ” 中 ， 他 在 某地...,1
1,2,警惕 看不到 内心 的 人 ， 虚荣 的 人 ， 心狠 的 人 ， 没有 是非观 的 人 ，...,1
2,3,这个 国在 计划生育 的 时候 对 人 都 能 下 杀手 ， 何况 现在 杭州 政策 杀狗 ...,1
3,4,罗伯特 . 所罗门 《 大 问题 — — 简明 哲学 导论 》 在 我们 这样 一个多元 的...,1
4,5,特朗普 ， 输 了 ！ O 特朗普 ， 输 了 ！,1


In [ ]:
test_submission = pd.DataFrame(test_pred_df)
test_submission.to_csv('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/submission.csv')

In [ ]:
import csv
csv.writer(open('/content/gdrive/MyDrive/4th_sem/dataset/Censorship/submission.tsv', 'w+'), delimiter='\t').writerows(csv.reader(open("/content/gdrive/MyDrive/4th_sem/dataset/Censorship/submission.csv")))